In [1]:
import os
import glob
import numpy as np
import time
from helper import R_to_angle
from params import par
from torchvision import transforms
from PIL import Image
import torch
import math

In [2]:
train_video = ['00', '02', '08', '09', '06', '04', '10']
# train_video = ['00', '02', '08' ]
image_path_list = []
for folder in train_video:
    image_path_list += glob.glob('KITTI/images/{}/*.png'.format(folder))

In [3]:
# image_path_list

In [4]:
len(image_path_list)

17437

In [5]:
img_as_img = Image.open(image_path_list[10])

In [6]:
to_tensor = transforms.ToTensor()

In [7]:
img_as_tensor = to_tensor(img_as_img)
img_as_tensor -= 0.5

In [8]:
img_as_tensor.max()

tensor(0.5000)

In [9]:
img_as_tensor.min()

tensor(-0.5000)

In [10]:
img_as_tensor.shape

torch.Size([3, 376, 1241])

In [11]:
img_as_np = np.array(img_as_img)
img_as_np = np.rollaxis(img_as_np, 2, 0)

In [12]:
img_as_np.shape

(3, 376, 1241)

In [13]:
img_as_np.max()

255

In [14]:
img_as_np.min()

0

In [15]:
img_as_tensor.std()

tensor(0.3237)

In [16]:
use_cuda = True
device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")


In [17]:
def denorm(batch, mean=[-0.0810, 0.1, 0.1], std=[0.3237, 0.4 , 0.4]):
    """
    Convert a batch of tensors to their original scale.

    Args:
        batch (torch.Tensor): Batch of normalized tensors.
        mean (torch.Tensor or list): Mean used for normalization.
        std (torch.Tensor or list): Standard deviation used for normalization.

    Returns:
        torch.Tensor: batch of tensors without normalization applied to them.
    """
    if isinstance(mean, list):
        mean = torch.tensor(mean).to(device)
    if isinstance(std, list):
        std = torch.tensor(std).to(device)

    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)

In [21]:
from data_helper import *
videos_to_test = ['07']

In [27]:
seq_len = int((par.seq_len[0]+par.seq_len[1])/2)
overlap = seq_len - 1
print('seq_len = {},  overlap = {}'.format(seq_len, overlap))
batch_size = par.batch_size

seq_len = 6,  overlap = 5


In [28]:
for test_video in videos_to_test:
    df = get_data_info(folder_list=[test_video], seq_len_range=[seq_len, seq_len], overlap=overlap, sample_times=1, shuffle=False, sort=False)

Folder 07 finish in 0.024933338165283203 sec


In [46]:
dataset = ImageSequenceDataset(df, par.resize_mode, (par.img_w, par.img_h), par.img_means, par.img_stds, par.minus_point_5)

In [61]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [67]:
for _,x,y in dataloader:
    print(x.shape, y.shape)
    print(x.mean(dims=(2,3,4)), x.std(dims=(2,3,4)))
    break

torch.Size([8, 6, 3, 608, 184]) torch.Size([8, 6, 6])


TypeError: mean() received an invalid combination of arguments - got (dims=tuple, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [65]:
par.img_means

(0.19007764876619865, 0.15170388157131237, 0.10659445665650864)

In [ ]:
par.img_m